In [1]:
!pip install glove_python_binary

     |████████████████████████████████| 948 kB 4.7 MB/s 


In [2]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

## 훈련 데이터 이해하기

In [3]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7f63420fd890>)

## 훈련 데이터 전처리하기

In [4]:
import nltk
nltk.download('punkt')

targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [6]:
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


# Glove 훈련시키기

In [7]:
from glove import Corpus, Glove

corpus = Corpus()

# 훈련 데이터로부터 Glove에서 사용할 동시 등장 행렬 생성
corpus.fit(result, window=5)
glove = Glove(no_components=100, learning_rate=0.05)

# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [9]:
print(glove.most_similar("man"))

[('woman', 0.9599452095811836), ('guy', 0.872086195542951), ('girl', 0.8679763587668655), ('boy', 0.8335846200140639)]


In [10]:
print(glove.most_similar("boy"))

[('girl', 0.937466367486514), ('woman', 0.8573826613745694), ('kid', 0.842096706103737), ('man', 0.8335846200140639)]


In [11]:
print(glove.most_similar("university"))

[('harvard', 0.886232050871184), ('mit', 0.8458138027776144), ('cambridge', 0.8350755628154798), ('stanford', 0.8283665684082245)]


In [12]:
print(glove.most_similar("water"))

[('air', 0.8438545104368603), ('fresh', 0.8386668953282582), ('clean', 0.8338102501753814), ('food', 0.816442338642196)]


In [13]:
print(glove.most_similar("physics"))

[('chemistry', 0.903653533963363), ('economics', 0.874698449442338), ('beauty', 0.8695892996369371), ('mathematics', 0.8689934556614323)]


In [14]:
print(glove.most_similar("muscle"))

[('tissue', 0.8593922629190488), ('nerve', 0.83628780427012), ('liquid', 0.7624917027040549), ('bone', 0.7622116420282391)]


In [15]:
print(glove.most_similar("clean"))

[('water', 0.8338102501753814), ('fresh', 0.8312030758168097), ('heat', 0.8064224035480806), ('wind', 0.7966821178215729)]
